In [1]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier

In [2]:
try:
    from catboost import CatBoostClassifier
except ImportError:
    print("CatBoost not found. Installing...")
    !pip install catboost
    from catboost import CatBoostClassifier

CatBoost not found. Installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling imports from your course list
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier # If not installed, use: !pip install catboost

# Evaluation and Preprocessing
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import balanced_accuracy_score, make_scorer, ConfusionMatrixDisplay

# Load datasets
train = pd.read_csv('job_change_train.csv')
test = pd.read_csv('job_change_test.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Target distribution:\n{train['willing_to_change_job'].value_counts(normalize=True)}")

Train shape: (12427, 15)
Test shape: (3308, 14)
Target distribution:
willing_to_change_job
No     0.751589
Yes    0.248411
Name: proportion, dtype: float64


In [5]:
def professional_preprocess(df):
    df = df.copy()

    # 1. Map Ordinal Categories to Numeric
    exp_map = {'<1': 0, '>20': 21, 'unknown': np.nan}
    change_map = {'never_changed': 0, '>4': 5, 'unknown': np.nan}

    df['years_of_experience'] = df['years_of_experience'].replace(exp_map).astype(float)
    df['years_since_job_change'] = df['years_since_job_change'].replace(change_map).astype(float)

    # 2. Impute Numeric with Median
    df['years_of_experience'] = df['years_of_experience'].fillna(df['years_of_experience'].median())
    df['years_since_job_change'] = df['years_since_job_change'].fillna(df['years_since_job_change'].median())

    # 3. Categorical Encoding (LabelEncoder is sufficient for tree-based models)
    cat_cols = df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        if col not in ['willing_to_change_job', 'id']:
            df[col] = df[col].fillna('Unknown')
            df[col] = LabelEncoder().fit_transform(df[col].astype(str))

    return df

# Apply preprocessing
train_cleaned = professional_preprocess(train)
X = train_cleaned.drop(['id', 'willing_to_change_job'], axis=1)
y = train['willing_to_change_job'].map({'Yes': 1, 'No': 0})

# Scale data for Distance-based models (KNN, SVM, LogReg)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
# 1. Final Training on all available data
final_model = CatBoostClassifier(silent=True, auto_class_weights='Balanced', iterations=500, depth=6)
final_model.fit(X, y)

# 2. Process Test Data
test_cleaned = professional_preprocess(test)
X_test = test_cleaned.drop(['id'], axis=1)
X_test_scaled = scaler.transform(X_test) # Use same scaler from training

# 3. Generate Predictions
test_preds = final_model.predict(X_test)

# 4. Format according to requirements (Only ID and Predicted Value)
submission = pd.DataFrame({
    'id': test['id'],
    'willing_to_change_job': pd.Series(test_preds).map({1: 'Yes', 0: 'No'})
})

submission.to_csv('final_predictions.csv', index=False)
print("Submission file 'final_predictions.csv' has been saved.")

Submission file 'final_predictions.csv' has been saved.
